In [2]:
import pandas as pd
from census import Census

# Census
census_key = "7e37dba12dd01d1c45b9d616212a92b3b667ae6b"

# create a list of years to loop through
census_combined = pd.DataFrame()

# 
for year in range(2010, 2019):
    # 
    c = Census(census_key, year=year)
    
    # 
    census_data = c.acs5.get(
        ("B01003_001E", "B19301_001E", "NAME", "GEO_ID"),
        {'for': 'county:*', 'in': 'state:02'}
    )
    
    # 
    census_pd = pd.DataFrame(census_data)
    census_pd['Year'] = year
    
    # 
    census_pd = census_pd.rename(
        columns={
            "B01003_001E": "Population",
            "B19301_001E": "Per Capita Income",
            "NAME": "County",
            "GEO_ID": "Zipcode",
            "state": "State"
        }
    )
    
    # 
    census_pd = census_pd[
        [
            "Year",
            "State",
            "County",
            "Population",
            "Per Capita Income",
            "Zipcode"
        ]
    ]
    
    # 
    census_combined = pd.concat([census_combined, census_pd])

# 
print(f"Number of rows in the DataFrame: {len(census_combined)}")
census_combined.head()


Number of rows in the DataFrame: 261


,Year,State,County,Population,Per Capita Income,Zipcode
0,2010,02,"Aleutians East Borough, Alaska",3703.0,22279.0,0500000US02013
1,2010,02,"Aleutians West Census Area, Alaska",6259.0,29920.0,0500000US02016
2,2010,02,"Anchorage Municipality, Alaska",284267.0,34678.0,0500000US02020
3,2010,02,"Bethel Census Area, Alaska",16838.0,18584.0,0500000US02050
4,2010,02,"Bristol Bay Borough, Alaska",1049.0,31260.0,0500000US02060


In [3]:
census_combined.head(15)

,Year,State,County,Population,Per Capita Income,Zipcode
0,2010,02,"Aleutians East Borough, Alaska",3703.0,22279.0,0500000US02013
1,2010,02,"Aleutians West Census Area, Alaska",6259.0,29920.0,0500000US02016
2,2010,02,"Anchorage Municipality, Alaska",284267.0,34678.0,0500000US02020
3,2010,02,"Bethel Census Area, Alaska",16838.0,18584.0,0500000US02050
4,2010,02,"Bristol Bay Borough, Alaska",1049.0,31260.0,0500000US02060
5,2010,02,"Denali Borough, Alaska",1144.0,42245.0,0500000US02068
6,2010,02,"Dillingham Census Area, Alaska",4817.0,22597.0,0500000US02070
7,2010,02,"Fairbanks North Star Borough, Alaska",94439.0,30395.0,0500000US02090
8,2010,02,"Haines Borough, Alaska",1658.0,27979.0,0500000US02100
9,2010,02,"Hoonah-Angoon Census Area, Alaska",2029.0,24932.0,0500000US02105


In [4]:
# clean the zip code column to only include the zip code
census_combined['Zipcode'] = census_combined['Zipcode'].str[-5:]
census_combined.head()

,Year,State,County,Population,Per Capita Income,Zipcode
0,2010,02,"Aleutians East Borough, Alaska",3703.0,22279.0,02013
1,2010,02,"Aleutians West Census Area, Alaska",6259.0,29920.0,02016
2,2010,02,"Anchorage Municipality, Alaska",284267.0,34678.0,02020
3,2010,02,"Bethel Census Area, Alaska",16838.0,18584.0,02050
4,2010,02,"Bristol Bay Borough, Alaska",1049.0,31260.0,02060


In [5]:
# cocatenate the population by year
total_population_by_year = census_combined.groupby('Year')['Population'].sum()
total_population_by_year


Year
2010    691189.0
2011    700703.0
2012    711139.0
2013    720316.0
2014    728300.0
2015    733375.0
2016    736855.0
2017    738565.0
2018    738516.0
Name: Population, dtype: float64

In [6]:
# Dependencies
import requests
import json

# get all factories in Alaska

latitude = 61.2181
longitude = -149.9003

categories = "production,production.factory,production.winery,production.brewery,production.cheese,production.pottery"
geoapify_key = "942a5446fd144eaf88ce6fd9245c5e95"
bias = f"proximity:{longitude},{latitude}"


params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": 500,
    "bias": bias
}

base_url = "https://api.geoapify.com/v2/places"

# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
# make  a new df with the factory name lat and long and zip code

factories_data = []


for feature in response.get("features", []):
    
    factory_name = feature["properties"].get("street", "N/A")

    
    categories = feature["properties"].get("categories", [])
    category = categories[0] if categories else "N/A"

    
    longitude, latitude = feature["geometry"]["coordinates"]

    
    postcode = feature["properties"].get("postcode", "N/A")

    
    factories_data.append({
        "Factory Name": factory_name,
        "Category": category,
        "Latitude": latitude,
        "Longitude": longitude,
        "Postcode": postcode
    })


factories_df = pd.DataFrame(factories_data)


print(factories_df)

                   Factory Name    Category   Latitude   Longitude Postcode
0               West 5th Avenue    catering  61.217681 -149.896978    99501
1               West 3rd Avenue    building  61.219764 -149.895809    99501
2                      G Street  production  61.208827 -149.895226    99501
3               Ocean Dock Road  production  61.230382 -149.893876    99501
4                Tidewater Road  production  61.234254 -149.884845    99501
5                Tidewater Road  production  61.235448 -149.883600    99501
6                 Denali Street  production  61.191450 -149.876337    99599
7            Old Seward Highway    building  61.190482 -149.869080    99503
8              West 58th Avenue  production  61.168644 -149.891641    99518
9                   King Street    catering  61.149129 -149.878652    99518
10             East 82nd Avenue  production  61.146573 -149.867821    99518
11           East Diamond Court  production  61.145305 -149.850835    99507
12          

In [ ]:
# Dependencies
import requests
import json

# get all factories in Alaska

latitude = 61.2181
longitude = -149.9003

categories = "tourism"
geoapify_key = "942a5446fd144eaf88ce6fd9245c5e95"
bias = f"proximity:{longitude},{latitude}"


params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": 500,
    "bias": bias
}

base_url = "https://api.geoapify.com/v2/places"

# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -149.89927200000002,
                    61.218678201737745
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "West 4th Avenue",
                "address_line2": "Anchorage, AK 99501, United States of America",
                "categories": [
                    "tourism",
                    "tourism.attraction",
                    "tourism.attraction.artwork"
                ],
                "city": "Anchorage",
                "country": "United States",
                "country_code": "us",
                "county": "Anchorage",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "artwork_type": "totem_pole",
                        "imag

In [ ]:
data = response.get("features", [])

tourism_data = []

for item in response.get("features", []):
    tourism_name = item["properties"].get("name", "N/A")
    category = item["properties"].get("categories", ["N/A"])[0]
    latitude = item["geometry"]["coordinates"][1]
    longitude = item["geometry"]["coordinates"][0]
    postcode = item["properties"].get("postcode", "N/A")

    tourism_data.append({
        "Tourism Name": tourism_name,
        "Category": category,
        "Latitude": latitude,
        "Longitude": longitude,
        "Postcode": postcode
    })

# 创建一个DataFrame
tourism_df = pd.DataFrame(tourism_data)

# 打印DataFrame
print(tourism_df)

                            Tourism Name  Category   Latitude   Longitude  \
0                                    N/A   tourism  61.218678 -149.899272   
1                                    N/A   tourism  61.218701 -149.899182   
2                                    N/A   tourism  61.217399 -149.898653   
3                  Captain Cook Monument  building  61.219311 -149.904056   
4                  Captain Cook Monument   tourism  61.219285 -149.903949   
..                                   ...       ...        ...         ...   
495                              Chugach  building  59.798603 -144.598795   
496            Cape St. Elias Lighthouse  building  59.798376 -144.598710   
497                              Chugach  building  59.798453 -144.598547   
498  Copper River & Northwestern Railway   tourism  61.438569 -143.718455   
499                     Gilahina Trestle   tourism  61.438659 -143.718516   

    Postcode  
0      99501  
1      99501  
2      99501  
3      99501  


In [ ]:
# Dependencies
import requests
import json

# get all factories in Alaska

latitude = 61.2181
longitude = -149.9003

categories = "commercial.outdoor_and_sport"
geoapify_key = "942a5446fd144eaf88ce6fd9245c5e95"
bias = f"proximity:{longitude},{latitude}"


params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": 500,
    "bias": bias
}

base_url = "https://api.geoapify.com/v2/places"

# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -149.8922993,
                    61.21630740173795
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "outfitters",
                "address_line2": "520 West 6th Avenue, Anchorage, AK 99501, United States of America",
                "categories": [
                    "commercial",
                    "commercial.outdoor_and_sport"
                ],
                "city": "Anchorage",
                "country": "United States",
                "country_code": "us",
                "county": "Anchorage",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "addr:housenumber": 520,
                        "addr:street": "West 6th Avenue",
     

In [ ]:
# Parse the JSON data
data = response.get("features", [])

# Extract the required information and store it in a list
outdoor_and_sport_data = []

for item in data:
    # Extract business name
    outdoor_and_sport_name = item["properties"].get("name", "N/A")
    
    # Extract categories and get the first category (if available)
    categories = item["properties"].get("categories", ["N/A"])
    category = categories[0]
    
    # Extract latitude and longitude
    latitude = item["geometry"]["coordinates"][1]
    longitude = item["geometry"]["coordinates"][0]
    
    # Extract postcode
    postcode = item["properties"].get("postcode", "N/A")

    # Append the extracted data to the list
    outdoor_and_sport_data.append({
        "Business Name": outdoor_and_sport_name,
        "Category": category,
        "Latitude": latitude,
        "Longitude": longitude,
        "Postcode": postcode
    })

# Create a DataFrame
outdoor_and_sport_df = pd.DataFrame(outdoor_and_sport_data)

# Print the DataFrame
print(outdoor_and_sport_df)

                     Business Name    Category   Latitude   Longitude Postcode
0    Sixth Avenue Outfitters Co-op  commercial  61.216307 -149.892299    99501
1                     Alaska eBike  commercial  61.200417 -149.907469    99503
2   Alaska Mountaineering & Hiking    building  61.196619 -149.904816    99503
3             Play It Again Sports  commercial  61.196567 -149.905978    99517
4                 The Bicycle Shop    building  61.195584 -149.902606    99503
..                             ...         ...        ...         ...      ...
85                  McBike & Sport  commercial  54.780271 -127.171193  V0J 3W0
86      BV Outdoor Essentials Ltd.  commercial  54.780531 -127.170382  V0J 3W0
87                       COB Bikes  commercial  54.778988 -127.171225  V0J 3W0
88                             N/A  commercial  54.517247 -126.838761      N/A
89                             N/A  commercial  54.375971 -126.749542      N/A

[90 rows x 5 columns]


In [ ]:
# Dependencies
import requests
import json

# get all factories in Alaska

latitude = 61.2181
longitude = -149.9003

categories = "activity"
geoapify_key = "942a5446fd144eaf88ce6fd9245c5e95"
bias = f"proximity:{longitude},{latitude}"


params = {
    "categories": categories,
    "apiKey": geoapify_key,
    "limit": 500,
    "bias": bias
}

base_url = "https://api.geoapify.com/v2/places"

# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -149.862048815036,
                    61.21292058600713
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Fairview Rec Center",
                "address_line2": "Karluk Street, Anchorage, AK 88591, United States of America",
                "categories": [
                    "activity",
                    "activity.community_center",
                    "building",
                    "building.public_and_civil"
                ],
                "city": "Anchorage",
                "country": "United States",
                "country_code": "us",
                "county": "Anchorage",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "ame

In [ ]:
# Parse the JSON data
data = response.get("features", [])

# Extract the required information and store it in a list
activity_data = []

for item in data:
    # Extract business name
    activity_name = item["properties"].get("name", "N/A")
    
    # Extract categories and get the first category (if available)
    categories = item["properties"].get("categories", ["N/A"])
    category = categories[0]
    
    # Extract latitude and longitude
    latitude = item["geometry"]["coordinates"][1]
    longitude = item["geometry"]["coordinates"][0]
    
    # Extract postcode
    postcode = item["properties"].get("postcode", "N/A")

    # Append the extracted data to the list
    activity_data.append({
        "Business Name": activity_name,
        "Category": category,
        "Latitude": latitude,
        "Longitude": longitude,
        "Postcode": postcode
    })

# Create a DataFrame
activity_df = pd.DataFrame(activity_data)

# Print the DataFrame
print(activity_df)

                                 Business Name        Category   Latitude  \
0                          Fairview Rec Center        activity  61.212921   
1                      Anchorage Senior Center        activity  61.202792   
2                            Arctic Gymnastics        activity  61.190229   
3                                     The Nave        activity  61.188757   
4                           Spenard Rec Center        activity  61.176491   
5                Asian Alaskan Cultural Center        activity  61.176841   
6           Mountainview Boy's and Girl's Club        activity  61.226239   
7                Arctic Oasis Community Center        activity  61.244745   
8                        Lidia Selkregg Chalet        activity  61.206130   
9                         Jim Haffey Ski Shack        activity  61.189377   
10       Anchorage Rowing Association Boat Pen        activity  61.152235   
11               Campbell Creek Science Center        activity  61.164100   

In [3]:
import hvplot.pandas

In [ ]:
# 为建筑数据创建图层，并设置透明度为0.5
tourism_plot = tourism_df.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    tiles='EsriImagery',  # 在每个图层中设置底图
    color='blue',
    size=5000,
    alpha=0.3,  # 设置透明度为0.5
    title="tourism"
)

# 为商业数据创建图层，并设置透明度为0.5
outdoor_and_sport_plot = outdoor_and_sport_df.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    tiles='EsriImagery',  # 在每个图层中设置底图
    color='green',
    size=5000,
    alpha=0.5,  # 设置透明度为0.5
    title="outdoor_and_sport"
)

# 为工厂数据创建图层，并设置透明度为0.5
factories_plot = factories_df.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    tiles='EsriImagery',  # 在每个图层中设置底图
    color='red',
    size=5000,
    alpha=0.3,  # 设置透明度为0.5
    title="Factories"
)

# 为活动数据创建图层，并设置透明度为0.5
activity_plot = activity_df.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    tiles='EsriImagery',  # 在每个图层中设置底图
    color='purple',
    size=5000,
    alpha=0.3,  # 设置透明度为0.5
    title="Activity"
)

# 叠加所有图层
combined_plot = (tourism_plot * outdoor_and_sport_plot * factories_plot * activity_plot).opts(
    frame_width=1200,  # 增大宽度
    frame_height=800   # 增大高度
)

# 显示地图
combined_plot


c:\Users\Sy X\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Points.I   :Points   [Longitude,Latitude]
   .WMTS.II    :WMTS   [Longitude,Latitude]
   .Points.II  :Points   [Longitude,Latitude]
   .WMTS.III   :WMTS   [Longitude,Latitude]
   .Points.III :Points   [Longitude,Latitude]
   .WMTS.IV    :WMTS   [Longitude,Latitude]
   .Points.IV  :Points   [Longitude,Latitude]

In [ ]:

import pandas as pd

tourism_json = tourism_df.to_json(orient='records', force_ascii=False)

print(tourism_json)

with open('tourism_data.json', 'w', encoding='utf-8') as file:
    file.write(tourism_json)


[{"Tourism Name":"N\/A","Category":"tourism","Latitude":61.2186782017,"Longitude":-149.899272,"Postcode":"99501"},{"Tourism Name":"N\/A","Category":"tourism","Latitude":61.2187011017,"Longitude":-149.899182,"Postcode":"99501"},{"Tourism Name":"N\/A","Category":"tourism","Latitude":61.2173985017,"Longitude":-149.8986531,"Postcode":"99501"},{"Tourism Name":"Captain Cook Monument","Category":"building","Latitude":61.2193109581,"Longitude":-149.9040563192,"Postcode":"99501"},{"Tourism Name":"Captain Cook Monument","Category":"tourism","Latitude":61.2192845017,"Longitude":-149.9039486,"Postcode":"99501"},{"Tourism Name":"N\/A","Category":"tourism","Latitude":61.2184285017,"Longitude":-149.895764,"Postcode":"99501"},{"Tourism Name":"Anchorage Light Speed Planet Walk","Category":"tourism","Latitude":61.2173304017,"Longitude":-149.8951977,"Postcode":"99501"},{"Tourism Name":"N\/A","Category":"tourism","Latitude":61.2181733017,"Longitude":-149.8932319,"Postcode":"99501"},{"Tourism Name":"Histor

In [ ]:

factories_json = factories_df.to_json(orient='records', force_ascii=False)


with open('factories_data.json', 'w', encoding='utf-8') as file:
    file.write(factories_json)


In [ ]:

outdoor_and_sport_json = outdoor_and_sport_df.to_json(orient='records', force_ascii=False)

with open('outdoor_and_sport_data.json', 'w', encoding='utf-8') as file:
    file.write(outdoor_and_sport_json)


In [ ]:

activity_json = activity_df.to_json(orient='records', force_ascii=False)

with open('activity_data.json', 'w', encoding='utf-8') as file:
    file.write(activity_json)


In [7]:
import pandas as pd

# 读取polar_bear_1985-2016.csv文件
polar_bear_df = pd.read_csv("polar_bear_1985-2016.csv")

# 更改列名为更简单的名称
new_column_names = {
    'BearID_mcp': 'BearID',
    'DateTimeUTC_mcp': 'DateTime',
    'latitude_mcp': 'Latitude',
    'longitude_mcp': 'Longitude'
}

polar_bear_df.rename(columns=new_column_names, inplace=True)

# 显示前几行数据以查看结果
polar_bear_df.head()


,BearID,DateTime,Latitude,Longitude
0,24,1985-05-06 23:29:00,70.584,-145.618
1,24,1985-05-08 22:29:00,70.618,-145.539
2,24,1985-05-09 03:23:00,70.616,-145.537
3,24,1985-05-10 22:44:00,70.640,-145.542
4,24,1985-05-12 00:15:00,70.663,-145.452


In [8]:
# find how many rows are in the data frame
print(f"Number of rows in the DataFrame: {len(polar_bear_df)}")

Number of rows in the DataFrame: 56977


In [9]:
polar_bear_df['DateTime'] = pd.to_datetime(polar_bear_df['DateTime'])

# 从DateTime列中提取年份
polar_bear_df['Year'] = polar_bear_df['DateTime'].dt.year

# 对年份进行分组，并计算每年的记录数
yearly_counts = polar_bear_df.groupby('Year').size()

# 打印结果
print(yearly_counts)

Year
1985     366
1986    1304
1987    1362
1988    2155
1989    3054
1990    1106
1991     558
1992    1838
1993    1842
1994     747
1997     148
1998     536
1999    1318
2000    1359
2001    2019
2002    1731
2003    1258
2004    3570
2005    4326
2006    4420
2007    2548
2008    2880
2009    4223
2010    1940
2011    2460
2012    2590
2013    2431
2014    1657
2015    1169
2016      62
dtype: int64


In [2]:
# 解析DateTime列为日期时间格式
polar_bear_df['DateTime'] = pd.to_datetime(polar_bear_df['DateTime'])

# 使用条件筛选出1985年的数据
df_1985_to_1990 = polar_bear_df[(polar_bear_df['DateTime'].dt.year >= 1985) & (polar_bear_df['DateTime'].dt.year <= 1990)]

# 显示前几行1985年的数据
df_1985_to_1990

print(type(df_1985_to_1990))


<class 'pandas.core.frame.DataFrame'>


In [3]:
# get how many rows are in the df
print(f"Number of rows in the DataFrame: {len(df_1985_to_1990)}")


Number of rows in the DataFrame: 9347


In [3]:
# 将DataFrame保存为JSON文件
df_1985_to_1990.to_json('polar_bear_1985_to_1990.json', orient='records', lines=True)


In [5]:
# 使用条件筛选出2011年到2016年之间的数据
df_2011_to_2016 = polar_bear_df[(polar_bear_df['DateTime'].dt.year >= 2011) & (polar_bear_df['DateTime'].dt.year <= 2016)]

# 显示前几行2011年到2016年之间的数据
df_2011_to_2016.head()

,BearID,DateTime,Latitude,Longitude
46608,236,2011-01-03 17:28:00,70.385,-135.159
46609,292,2011-01-04 18:58:00,74.267,-164.612
46610,236,2011-01-06 20:15:00,70.558,-137.083
46611,292,2011-01-07 22:08:00,74.275,-166.081
46612,292,2011-01-08 01:28:00,74.273,-166.089


In [6]:
# print how many rows are in the df
print(f"Number of rows in the DataFrame: {len(df_2011_to_2016)}")

Number of rows in the DataFrame: 10369


In [6]:
df_2011_to_2016.to_json('polar_bear_2011_to_2016.json', orient='records', lines=True)

In [22]:
# 为工厂数据创建图层，并设置透明度为0.5
df_1985_to_1990 = df_1985_to_1990.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    tiles='EsriImagery',  # 在每个图层中设置底图
    color='green',
    size=50,
    alpha=0.3,  # 设置透明度为0.5
    title="df_1985_to_1990"
)

# 为活动数据创建图层，并设置透明度为0.5
df_2011_to_2016 = df_2011_to_2016.hvplot.points(
    'Longitude', 
    'Latitude', 
    geo=True, 
    tiles='EsriImagery',  # 在每个图层中设置底图
    color='red',
    size=50,
    alpha=0.3,  # 设置透明度为0.5
    title="df_2011_to_2016"
)

# 叠加所有图层
combined_plot = (df_2011_to_2016 * df_1985_to_1990).opts(
    frame_width=1200,  # 增大宽度
    frame_height=800   # 增大高度
)

# 显示地图
combined_plot

:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]
   .WMTS.II   :WMTS   [Longitude,Latitude]
   .Points.II :Points   [Longitude,Latitude]